In [ ]:
%matplotlib widget

import jscatter as js
import numpy as np
print(js.version)

# Alcohol dehydrogenase (yeast) example for normal mode analysis

See Biehl et al. Direct Observation of Correlated Interdomain Motion in Alcohol Dehydrogenase (https://doi.org/10.1103/PhysRevLett.101.138102)


### nglview for viewing universes

In [ ]:
# nglview for viewing universes
import nglview as nv

# for additional MDAnalysis funtionality if needed
# import MDAnalysis as mda

### create universe with protein inside that adds hydrogens
 - load PDB structure
 - repair structure e.g. missing atoms 
 - add hydrogens using pdb2pqr, saving this to 3cln_h.pdb
 - adds scattering amlitudes, volume determination
Select what you needhere 'protein'

In [ ]:
adh = js.bio.fetch_pdb('4w6z.pdb1')
# the 2 dimers in are in model 1 and 2 and need to be merged into one.
adhmerged = js.bio.mergePDBModel(adh)
u=js.bio.scatteringUniverse(adhmerged)

# select protein atoms
protein = u.select_atoms("protein").residues

### look at it

In [ ]:
# create viewer
w = nv.show_mdanalysis(u.atoms)
# show it
w

### calc neutron scattering  of the protein
- use protonated and deuterated protein

coarse graining (cubesize=2) is only for speedup the example

In [ ]:
u.qlist = js.loglist(0.1,5,100) 
u.setSolvent(['55.5d2o1','0h2o1','0.15Na1Cl1'],temperature=273+5)

u.atoms.deuteration = 0
# calc SANS scattering without surface
S0 = js.bio.nscatIntUniv(protein,cubesize=2)

u.atoms.deuteration = 1
S1 = js.bio.nscatIntUniv(protein,cubesize=2)

print(S0.columnname)

S0

In [ ]:
# plot it
p=js.mplot()
p.Plot(S0)
p.Plot(S1)
p.Xaxis(label='$Q\, /\, nm^{-1}$',min=0.1,max=5)
p.Yaxis(label='$F(Q)\, /\, nm^{2}/particle $',min=1e-7,max=0.004)

In [ ]:
u.segments.segids
u.atoms.deuteration = 0
u.select_atoms('segid C').atoms.deuteration=1
S2 = js.bio.nscatIntUniv(protein,cubesize=2)

p.Plot(S2,le='partial deuteration')
p.Xaxis(label='$Q\, /\, nm^{-1}$',min=0.1,max=5)
p.Yaxis(label='$F(Q)\, /\, nm^{2}/particle $',min=1e-7,max=0.004)

## Create normal modes for Ca atoms
- create normal modes
- make trajectory like a MD simulation
- set embedding solvent 
- calc scattering

In [ ]:
u.atoms.deuteration = 0
ca = u.residues
nm = js.bio.vibNM(ca)


## Calc effective diffusion with trans/rot contributions
- determine $D_{trans}$ and $D_{rot}$ using HULLRAD
- calc Deff

$\begin{split}D_0(Q) = \frac{1}{Q^2F(Q)} \sum_{j,k}
\langle b_je^{-iQr_j} \begin{pmatrix} Q \\ r_j \times Q \end{pmatrix} D_{6x6}
\begin{pmatrix} Q \\ r_j \times Q \end{pmatrix}  b_ke^{-iQr_k}
\rangle\end{split}$

In [ ]:
D_hr = js.bio.hullRad(u)
Dt = D_hr['Dt'] * 1e2  # conversion to nm²/ps
Dr = D_hr['Dr'] * 1e-12  # conversion to 1/ps
u.qlist = np.r_[0.01, 0.1:3:0.2]
Deff = js.bio.diffusionTRUnivTensor(u.residues, DTT=Dt, DRR=Dr,cubesize=2)


In [ ]:
p=js.mplot()
p.Plot(Deff.X,Deff.Y*1e5,li=1)
p.Xaxis(label='$Q / nm^{-1}$')
p.Yaxis(label='$D_{eff} /A^2/ns$')

### Normal mode relaxation in small displacement approximation

## $I(Q,t) =F(Q) + \sum_{\alpha}e^{-\lambda_{\alpha} t}P_{\alpha}(Q)$

## $P_{\alpha}(Q) = \Big \langle \sum_{k,l} b_kb_l e^{iQ(r_k-r_l)}
(\vec{Q}*\vec{d}_{\alpha,k}) (\vec{Q}*\vec{d}_{\alpha,l}) \Big \rangle$

 ### mode formfactors in small displacement approximation

We use $\vec{d}_{\alpha,l} = a_{l,\alpha} \hat{v}_{\alpha,l}$ 
with weighted modes $a_{l,\alpha} =\sqrt{\frac{kT}{k_{\alpha}}} = \sqrt{\frac{kT}{m_l\omega^2_{\alpha}}}$



In [ ]:
Ph678 = js.dL()
for NN in [6,7,8]:
   Ph = js.bio.intScatFuncPMode(nm, NN,output=0,qlist=Deff.X)
   Ph678.append(Ph)


## effective Diffusion in initial slope (cumulant fit)
## $D(Q) = D_{t,r}(Q) + \Delta D_{eff}(Q)$

## $\Delta D_{eff}(Q) =
\lambda_{\alpha} a_{\alpha}^2 P_{\alpha}(Q) / ( Q^2[F(Q)+a_{\alpha}^2 P_{\alpha}(Q)])$

In [ ]:
a=1000.
rate = 1/30000 # 1/ps
for Ph in Ph678:
   d =  Deff.interp(Ph.X) + rate * a**2 * Ph._Pn / (Ph._Fq+a**2*Ph._Pn) / Ph.X**2
   p.Plot(Ph.X,1e5*d ,li='', le=f'mode {Ph.modeNumber}  rmsd={Ph.kTrmsd*a:.2f} A')
p.legend()

## Assume a common relaxation on top of diffusion

### Diffusion in Rayleigh expansion with scalar $D_{trans}, D_{rot}$

### $\frac{I(Q,t)}{I(Q,0)}_{t,r} = e^{-D_{trans}Q^2t}\sum_l S_l(Q)e^{-l(l+1)D_{rot}t} /  \sum_l S_l(Q)$

$S_l(Q) = \sum_m \bigg| \sum_j b_j(Q)j_l(Qr_j) Y_{lm}(\Omega_j)  \bigg|^2$

In [ ]:
u.qlist = np.r_[0.2:2:0.2]      # [1/nm]
u.tlist = np.r_[1,10:1e5:50]  # [ps]

Iqt = js.bio.intScatFuncYlm(u.residues,Dtrans=Dt,Drot=Dr,cubesize=1,getVolume='once')

p=js.mplot()
p.Plot(Iqt,sy=0,li=1)
p.Yaxis(min=0.01,max=1,label='I(Q,t)/I(Q,0)',scale='log')
p.Xaxis(label='t / ps')


### Mode relaxation in small displacement approximation

## $\frac{I(Q,t)}{I(Q,0)}_{int} = 
\frac{F(Q) + \sum_{\alpha}e^{-\lambda_{\alpha} t}P_{\alpha}(Q)}{F(Q) + \sum_{\alpha}P_{\alpha}(Q)} = 
\frac{F(Q) + e^{-\lambda_{\alpha} t}a_{\alpha}^2 \hat{P}_{\alpha}(Q)}{F(Q) + a_{\alpha}^2 \hat{P}_{\alpha}(Q)}$

## $= (1-A(Q))+A(Q)e^{-\lambda t}$  with $A(Q) = \frac{a_{\alpha}^2 \hat{P}_{\alpha}(Q)}{F(Q) + a_{\alpha}^2 \hat{P}_{\alpha}(Q)}$

### together


### $\frac{I_1(Q,t)}{I_1(Q,t)} = 
e^{-D_{trans}Q^2t}\frac{\sum_l S_l(Q)e^{-l(l+1)D_{rot}t}}{\sum_l S_l(Q)} [(1-A(Q) + A(Q)e^{-\lambda t}] $

In [ ]:
sumP = Ph678.Y.array.sum(axis=0)

def Aq(a):
    # NM mode formfactor sum
    aq = a**2*sumP / (Ph678[0]._Fq + a**2*sumP )
    return js.dA(np.c_[Ph678[0].X, aq].T)


p2=js.mplot()
p2.Yaxis(min=0.01,max=1,label='I(Q,t)/I(Q,0)')
p2.Xaxis(min=0,max=100000,label='t / ps')

Iqt2 = Iqt.copy()
l=1/10000  # 1/ps
Aqa = Aq(a)
for i,qt in enumerate(Iqt2):
    diff = qt.Y *(1-Aqa.interp(qt.q))
    qt.Y = qt.Y *((1-Aqa.interp(qt.q)) + Aqa.interp(qt.q)*np.exp(-l*qt.X))
     
    p2.Plot(qt.X,qt.Y * 0.8**i,sy=0,li=[3,2,i+1],le=f'{qt.q:.1f}')
    p2.Plot(qt.X,diff* 0.8**i,sy=0,li=[1,2,i+1])
    
p2.Yaxis(min=0.001,max=1,label='I(Q,t)/I(Q,0)',scale='log')
p2.Xaxis(min=0.1,max=100000,label='t / ps')
p2.Legend()

In [ ]:
p1=js.mplot()
Aqa = Aq(a)
p1.Plot(Aqa,sy=0,li=1)
p1.Yaxis(min=0,max=0.8)

# A first fit model
## Include H(Q) and S(Q)

In [ ]:
# structure factorfrom SAXS/SANS
q = js.loglist(0.1,4,100)
Sq = js.sf.PercusYevick(q,4,eta=0.05) # about 50mg/ml protein 
p3 = js.mplot()
p3.plot(Sq)
# correction ala Kotlarchyk
Sq.Y = 1+S0.interp(Sq.X,col='beta')*(Sq.Y-1)  
p3.plot(Sq)


## Include  $D_t = D_0 H(Q)/S(Q)$ correction

In [ ]:
sumP = Ph678.Y.array.sum(axis=0)
def Aq(a):
    aq = a**2*sumP / (Ph678[0]._Fq + a**2*sumP )
    return js.dA(np.c_[Ph678[0].X, aq].T)


def transRotModes(q,Dt,Dr,a=1000,H=1,l=10):
    # trans rot diffusion including H/S(Q)
    # assume H constant, 1-Hr =(1-Ht)/3 
    Dth = Dt*H/Sq.interp(q)
    Drh = Dr*(1-(1-H)/3)
    Iqt = js.bio.intScatFuncYlm(u.residues,Dtrans=Dth,Drot=Drh,cubesize=1,qlist=np.r_[q],getVolume='once')[0]
    
    # add Pmode relaxation
    Aqa = Aq(a)
    diff = Iqt.Y *(1-Aqa.interp(q))
    Iqt.Y = Iqt.Y *((1-Aqa.interp(q)) + Aqa.interp(q)*np.exp(-1/(l*1000)*Iqt.X))
    Iqt2 = Iqt.addColumn(1,diff)
    
    # for reference in lastfit
    Iqt2.Dt = Dt
    Iqt2.Dr = Dr
    Iqt2.H = H
    return Iqt2

In [ ]:
sim = js.dL()
for q in np.r_[0.5,0.9,1.2]:
    sim.append(transRotModes(q,Dt,Dr,a,1,10))

In [ ]:
p4=js.mplot()
for si in sim:
    p4.plot(si,sy=0,li=1)
    p4.plot(si.X,si[2],sy=0,li=[3,1,1])
p4.Yaxis(min=0.01,max=1,label='I(Q,t)/I(Q,0)',scale='log')
p4.Xaxis(min=0.1,max=100000,label='t / ps')